<a href="https://colab.research.google.com/github/DrakeShadowRaven/desihigh/blob/main/n'ish_body.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/desihigh')

In [4]:
import time
import astropy
import itertools
import matplotlib

import numpy             as np
import pylab             as pl
import matplotlib.pyplot as plt
import astropy.units     as u

from   astropy.cosmology import FlatLambdaCDM
from   IPython.display   import YouTubeVideo
from   tools.flops       import flops

In [5]:
from IPython.display import clear_output
from time import sleep

In [6]:
%matplotlib inline

plt.style.use('dark_background')

In [7]:
def init_dof(npt=1):
  #  Create a set of particles at random positions in a box, which will soon predict the distribution of dark matter 
  #  as we see above.
  xs     = np.random.uniform(0., 1., npt)
  ys     = np.random.uniform(0., 1., npt)
  zs     = np.random.uniform(0., 1., npt)

  pos    = np.vstack((xs,ys,zs)).T
  vel    = np.zeros_like(pos)

  return  pos, vel

In [19]:
# Reinitialise particles.
npt=2
pos, vel = init_dof(npt=npt)
mass = np.random.uniform(0., 1., npt)
msize = mass*20
print, mass

# force the last mass very small


print,mass
# pos of partile 1

i = 0
pos[i][0]=0.5
pos[i][1]=0.5
pos[i][2]=0.5
pos[0]=0

i = 1 
pos[i][0]=0.7
pos[i][1]=0.5
pos[i][2]=0.5




# vel of partile 1

i = 0
vel[i][0]=0.1
vel[i][1]=0.05
vel[i][2]=0.1
vel[0]=11000

i = 1 
vel[i][0]=0.05
vel[i][1]=0.1
vel[i][2]=0.2
vel[1]=-8250

mass[0]=6e30
mass[1]=1.5e30


In [11]:
print(pos)

[[0.5        0.5        0.5       ]
 [0.7        0.5        0.5       ]
 [0.7        0.57       0.5       ]
 [0.01887783 0.04233641 0.87437312]
 [0.06407237 0.2313325  0.08580478]
 [0.87461356 0.95940627 0.97530755]
 [0.69475581 0.98191221 0.06007324]
 [0.94140468 0.94076411 0.84332235]
 [0.76237747 0.37610145 0.58100756]
 [0.28619212 0.65216499 0.6636948 ]]
